<a href="https://colab.research.google.com/github/knh0503/CrimeManagementSystem/blob/main/crime_predictor_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSV 파일 준비

1. 테이블을 dump 하여 만든 csv 파일을 구글 드라이브에

2. 코랩에 드라이브를 마운트하여 csv 파일을 사용할 수 있도록 준비

3. csv 파일 경로를 복사하여 csv 파일을 읽어 데이터 준비

# 그룹화 및 crime_count 칼럼 추가

각 지역마다 시간대별 범죄 횟수를 카운트하여 새로운 칼럼으로 추가해준다.

1. df.groupby(['region', 'month', 'week', 'day_of_week'])
  - groupby는 데이터를 지정된 컬럼을 기준으로 그룹화

2. .size()
  - size()는 그룹별 데이터의 크기(개수)를 계산

3. .reset_index(name='crime_count')
  - 계산 결과를 새로운 데이터프레임으로 변환하고, 컬럼 이름을 crime_count로 설정

In [ ]:
import pandas as pd

# 파일 경로 지정
file_path = '/content/drive/MyDrive/files/crime_time_region_data.csv'

# CSV 파일 읽기
data = pd.read_csv(file_path)

# 그룹화하여 범죄 횟수 계산
grouped_data = data.groupby(['region', 'month', 'week', 'day_of_week']).size().reset_index(name='crime_count')

# 데이터 확인
print(grouped_data.head())  # 첫 5행 출력

   region  month  week day_of_week  crime_count
0     0.0      1     2   Wednesday            1
1     0.0      2     2     Tuesday            1
2     0.0      2     3      Sunday            1
3     0.0      3     1      Monday            1
4     0.0      3     4      Friday            1


# day_of_week 데이터 인코딩

머신러닝 모델을 사용하기 위해서는 Monday와 같은 str변수를 int로 변환해줘야 한다.

이를 위해 dict에 원하는 순서대로 저장한후 map을 사용하여 매핑을 진행해준다.

In [ ]:
# 요일을 원하는 순서로 인코딩 (일월화수목금토 = 0123456)
day_mapping = {
    'Sunday' : 0,
    'Monday' : 1,
    'Tuesday' : 2,
    'Wednesday' : 3,
    'Thursday' : 4,
    'Friday' : 5,
    'Saturday' : 6
}

# 매핑 적용
grouped_data['day_of_week_encoded'] = grouped_data['day_of_week'].map(day_mapping)


# 훈련 세트, 데이터 세트 분할

In [ ]:
from sklearn.model_selection import train_test_split

X = grouped_data[['region', 'month', 'week', 'day_of_week_encoded']]
y = grouped_data['crime_count']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# HistGradientBoostingRegressor 모델 정의 및 훈련

히스토그램 기반 그래디언트부스팅 모델은 확률적 경사 하강법을 응용하여, 깊이가 얕은 트리를 사용하여 이전 트리를 보완하는 형식으로 앙상블하는 머신러닝 모델이다. 입력값을 히스토그램으로 변환하기 위하여 input을 256개의 구간으로 나눈다.

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

# HistGradientBoosting 모델 정의 및 학습
model = HistGradientBoostingRegressor(
    max_iter=500,                        # 부스팅 스테이지 수(에포크 횟수)
    max_depth=5,                        # 트리 최대 깊이
    min_samples_leaf=5,                       # 리프 노드의 최소 샘플 수
    learning_rate=0.05,                   # 학습률
    l2_regularization=1.0,               # L2 정규화 강도
    random_state=42
)

# 모델 학습
model.fit(X_train, y_train)

HistGradientBoostingRegressor(l2_regularization=1.0, learning_rate=0.05,
                              max_depth=5, max_iter=500, min_samples_leaf=5,
                              random_state=42)

# 모델 평가

## cross_validate

In [ ]:
from sklearn.model_selection import cross_validate
import numpy as np

scores = cross_validate(model, X_train, y_train,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9618979534399494 0.957316539668754


In [ ]:
from sklearn.model_selection import KFold, cross_val_score, cross_validate

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores_cv = cross_val_score(model, X, y, cv=kf)
scores_cval = cross_validate(model, X_train, y_train, cv=kf, return_train_score=True)

print("cross_val_score:", np.mean(scores_cv))
print("cross_validate (train):", np.mean(scores_cval['train_score']))
print("cross_validate (test):", np.mean(scores_cval['test_score']))


cross_val_score: 0.957125229996227
cross_validate (train): 0.9615993143136145
cross_validate (test): 0.9572806080936133


## 예측 함수 정의

In [ ]:
def predict_crime_count(region, month, week, day_of_week):

    # 요일을 인코딩된 숫자로 변환
    day_encoded = day_mapping[day_of_week]

    # 예측 수행
    prediction = model.predict([[region, month, week, day_encoded]])[0]
    return round(prediction, 2)

In [ ]:
# 예측 예시
print("\n예측 예시:")
example_cases = [
    (1, 11, 4, "Sunday"),
    (30, 1, 1, "Monday"),
    (70, 1, 15, "Friday")
]

for case in example_cases:
    region, month, week, day = case
    predicted = predict_crime_count(region, month, week, day)
    print(f"Region: {region}, Month: {month}, Week: {week}, Day: {day}")
    print(f"예측된 범죄 횟수: {predicted}\n")


예측 예시:
Region: 1, Month: 11, Week: 4, Day: Sunday
예측된 범죄 횟수: 45.06

Region: 30, Month: 1, Week: 1, Day: Monday
예측된 범죄 횟수: 63.14

Region: 70, Month: 1, Week: 15, Day: Friday
예측된 범죄 횟수: 19.13



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(


## 모델 성능 평가

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

def evaluate_model():
    # 학습 데이터에 대한 예측
    y_train_pred = model.predict(X_train)

    # 테스트 데이터에 대한 예측
    y_test_pred = model.predict(X_test)

    # R2 점수 계산
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # RMSE 계산 (mean_squared_error)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    # 교차 검증 점수
    cv_scores = cross_val_score(model, X, y, cv=5)

    print("모델 성능 평가:")
    print(f"Train R² Score: {train_r2:.4f}")
    print(f"Test R² Score: {test_r2:.4f}")
    print(f"Train RMSE: {train_rmse:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")
    print(f"5-Fold CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

In [ ]:
evaluate_model()

모델 성능 평가:
Train R² Score: 0.9614
Test R² Score: 0.9573
Train RMSE: 8.5365
Test RMSE: 8.9418
5-Fold CV Score: -0.3507 (+/- 0.5500)


### 5-Fold CV Score 점수 이상

R2, RMSE 점수는 괜찮지만 5-fold cv score가 음수가 나왔다. 정상적이라면 R2 점수와 비슷해야 한다. 해결을 위하여 전체 세트, 훈련 세트, 테스트 세트 각각에 대하여 교차 검증을 실시했다.

In [ ]:
print(cross_val_score(model, X, y, cv=5))
print(cross_val_score(model, X_train, y_train, cv=5))
print(cross_val_score(model, X_test, y_test, cv=5))

[ 0.09981474 -0.2299464  -0.63918939 -0.3555572  -0.62866613]
[0.95842408 0.95674383 0.95989023 0.95560963 0.95591492]
[0.95442285 0.93543786 0.96037149 0.95483375 0.95161919]


훈련 세트. 테스트 세트에 대하여 교차 검증을 수행하면 점수가 정상적이지만 전체 세트에 대해서만 점수가 이상하다는 것을 확인하였다.

이를 해결하기 위해 X, X_train, X_test와 y, y_train, y_test에 대하여 차이점을 확인해본다.

In [ ]:
# 데이터 크기 확인
print("X shape:", X.shape)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

# 데이터 분포 확인
print("\nX의 처음 몇 개 샘플:")
print(X.head(10))
print("\nX_train의 처음 몇 개 샘플:")
print(X_train.head(10))

# 데이터 관측
print("\nX의 기술통계")
print(X.describe())
print("\nX_train의 기술통계")
print(X_train.describe())
print("\nX_test의 기술통계")
print(X_test.describe())

X shape: (31876, 4)
X_train shape: (25500, 4)
X_test shape: (6376, 4)

X의 처음 몇 개 샘플:
   region  month  week  day_of_week_encoded
0     0.0      1     2                    3
1     0.0      2     2                    2
2     0.0      2     3                    0
3     0.0      3     1                    1
4     0.0      3     4                    5
5     0.0      5     1                    5
6     0.0      5     3                    1
7     0.0      5     4                    5
8     0.0      6     4                    0
9     0.0     10     2                    0

X_train의 처음 몇 개 샘플:
       region  month  week  day_of_week_encoded
28290    69.0      5     1                    2
1592      4.0     10     4                    0
13088    32.0      8     3                    4
29244    71.0      8     4                    0
29653    72.0      8     3                    3
24191    59.0      6     3                    2
17        1.0      1     1                    4
5712     14.0     11     1

In [ ]:
print("y의 기술통계:")
print(y.describe())
print("\ny_train의 기술통계:")
print(y_train.describe())
print("\ny_test의 기술통계:")
print(y_test.describe())

y의 기술통계:
count    31876.000000
mean        45.691994
std         43.425468
min          1.000000
25%         15.000000
50%         30.000000
75%         65.000000
max        341.000000
Name: crime_count, dtype: float64

y_train의 기술통계:
count    25500.000000
mean        45.754314
std         43.465963
min          1.000000
25%         15.000000
50%         30.000000
75%         65.000000
max        341.000000
Name: crime_count, dtype: float64

y_test의 기술통계:
count    6376.000000
mean       45.442754
std        43.265635
min         1.000000
25%        15.000000
50%        30.000000
75%        64.000000
max       328.000000
Name: crime_count, dtype: float64


전체 세트, 훈련 세트, 테스트 세트에 대한 기술 통계는 별 차이가 없는 것을 확인하였다.

그러나 head()를 사용하여 처음 5개의 샘플을 출력하니 region이 모두 0으로 나왔다. 이는 교차검증을 전체 세트에 대하여 수행할 때 정보의 불균형을 일으킬 수 있다. 기본적으로 shuffle=False이므로 데이터를 섞지 않고 교차검증을 수행하기 때문이다.

### 점수 이상 원인 분석

In [ ]:
# 교차 검증 점수
cv_scores = cross_val_score(model, X, y, cv=5)

위의 코드에서 cv=5를 설정하였는데 이는 kFold(n_splits=5)와 동일하며 shuffle=False로 되어있다. 따라서 X,y에 대하여 교차검증을 할 때 데이터를 섞지 않고 순서대로 5개의 폴드를 나누어 교차 검증이 수행된다.

하지만 X를 head를 이용하여 조사하면 0부터 77까지 순서대로 되어 있으므로 데이터가 편향되는 경향을 보이는 것이다.

참고로 X_train, X_test의 경우는 train_test_split 함수에서 미리 섞고 훈련 세트, 테스트 세트를 분할하기 때문에 교차 검증에서 정보 편향이 되지 않는다.

In [ ]:
print(X.head)

<bound method NDFrame.head of        region  month  week  day_of_week_encoded
0         0.0      1     2                    3
1         0.0      2     2                    2
2         0.0      2     3                    0
3         0.0      3     1                    1
4         0.0      3     4                    5
...       ...    ...   ...                  ...
31871    77.0     12     5                    6
31872    77.0     12     5                    0
31873    77.0     12     5                    4
31874    77.0     12     5                    2
31875    77.0     12     5                    3

[31876 rows x 4 columns]>


즉 이를 해결 하려면 교차 검증을 할 때 데이터를 섞은 후에 교차 검증을 하면 된다.

### 문제 해결

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold

def evaluate_model():
    # 학습 데이터에 대한 예측
    y_train_pred = model.predict(X_train)

    # 테스트 데이터에 대한 예측
    y_test_pred = model.predict(X_test)

    # R2 점수 계산
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # RMSE 계산 (mean_squared_error)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    # 교차 검증 점수
    #cv_scores = cross_val_score(model, X, y, cv=5)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X, y, cv=kf)

    print("모델 성능 평가:")
    print(f"Train R² Score: {train_r2:.4f}")
    print(f"Test R² Score: {test_r2:.4f}")
    print(f"Train RMSE: {train_rmse:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")
    print(f"5-Fold CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

In [ ]:
evaluate_model()

모델 성능 평가:
Train R² Score: 0.9614
Test R² Score: 0.9573
Train RMSE: 8.5365
Test RMSE: 8.9418
5-Fold CV Score: 0.9571 (+/- 0.0010)


5-Fold CV Score가 정상적으로 출력되었다.

## 특성 중요도 분석

In [ ]:
from sklearn.inspection import permutation_importance

# 특성 중요도 계산
result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)

# 특성 중요도 출력
importance = result.importances_mean
for i, col in enumerate(X.columns):
    print(f"Feature: {col}, Importance: {importance[i]}")

Feature: region, Importance: 1.6631859792436852
Feature: month, Importance: 0.02783339880650574
Feature: week, Importance: 0.30545935542092534
Feature: day_of_week_encoded, Importance: 0.013295641307261973


region -> week -> month -> day_of_week 순으로 특성 중요도가 높은 것을 확인할 수 있다.

## 요일별 통계 분석

In [ ]:
def analyze_day_stats(grouped_data):
    # 요일별 범죄 통계 계산
    day_stats = (
        grouped_data.groupby('day_of_week')['crime_count']
        .agg(['mean', 'count', 'std'])
        .round(2)
    )

    # 요일 순서 정의 및 정렬
    day_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
    day_stats = day_stats.loc[day_order]

    print("\n요일별 범죄 통계")
    print(day_stats)

In [ ]:
analyze_day_stats(grouped_data)


요일별 범죄 통계
              mean  count    std
day_of_week                     
Sunday       44.60   4533  42.55
Monday       44.67   4606  42.28
Tuesday      45.43   4536  42.92
Wednesday    45.23   4606  43.26
Thursday     45.44   4529  42.82
Friday       48.22   4534  45.33
Saturday     46.27   4532  44.67


금요일에 범죄가 가장 많이 일어나고, 월요일에 가장 적게 일어나는 것을 확인할 수 있다.

# GridSearchCV로 최적의 파라미터 찾기

In [ ]:
# 1. GridSearchCV로 전체 데이터에 대한 최적의 파라미터 찾기
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_iter': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 10],
    'learning_rate': [0.05, 0.1, 0.2]
}

grid_search = GridSearchCV(model, param_grid, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best CV score:", grid_search.best_score_)

Best parameters: {'learning_rate': 0.2, 'max_depth': 10, 'max_iter': 300}
Best CV score: 0.9615243486020872


In [ ]:
dt = grid_search.best_estimator_
print(dt.score(X_test, y_test))

0.9621395595682536


하이퍼 파라미터 튜닝을 위해 GridSearchCV 클래스를 사용하였다.

최적의 파라미터를 사용해 학습된 모델은 grid_search.best_params 에 저장되어 있으므로 이를 저장한다.

# 완성된 모델로 예측

In [ ]:
def predict_crime_count(region, month, week, day_of_week):

    # 요일을 인코딩된 숫자로 변환
    day_encoded = day_mapping[day_of_week]

    # 예측 수행
    prediction = dt.predict([[region, month, week, day_encoded]])[0]
    return round(prediction, 2)

In [ ]:
# 예측 예시
print("\n예측 예시:")
example_cases = [
    (1, 11, 4, "Sunday"),
    (30, 1, 1, "Monday"),
    (70, 1, 15, "Friday")
]

for case in example_cases:
    region, month, week, day = case
    predicted = predict_crime_count(region, month, week, day)
    print(f"Region: {region}, Month: {month}, Week: {week}, Day: {day}")
    print(f"예측된 범죄 횟수: {predicted}\n")


예측 예시:
Region: 1, Month: 11, Week: 4, Day: Sunday
예측된 범죄 횟수: 45.1

Region: 30, Month: 1, Week: 1, Day: Monday
예측된 범죄 횟수: 59.06

Region: 70, Month: 1, Week: 15, Day: Friday
예측된 범죄 횟수: 24.4



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(


# 모델 저장

In [ ]:
import joblib

# 최적 모델을 저장
joblib.dump(grid_search.best_estimator_, 'crime_predictor_model.pkl')